# INTRO

In [6]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', None)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# IMPORT CSV _ lista de doutores

In [7]:
df = pd.read_csv('doutoresFeaturesParte_1.csv', encoding='UTF-8', nrows=200) #, dtype=str, nrows=1000)

df = df.fillna(' ')

df = df.astype({
    'IDLattes': int,
    'DuracaoDoutorado': float,
    'Genero': str,
    'GrandesAreas': str,
    'ExclusividadeDoutorado': str,
    'Publicacoes': int,
    'IdadeAcademica': str,
    'ListaFiliacoes': str,
    'UltimaFiliacao': str,
})

df2 = df.copy(deep=True)
df2

,IDLattes,DuracaoDoutorado,Genero,GrandesAreas,ExclusividadeDoutorado,Publicacoes,IdadeAcademica,ListaFiliacoes,UltimaFiliacao
0,39015885890,4.0,,,,9,9.0,UNIVERSIDADE FEDERAL DO ABC; INSTITUTO DE FISI...,INSTITUTO DE ASTRONOMIA GEOFISICA E CIENCIAS A...
1,104116232000,4.0,,,,8,5.0,UNIVERSIDADE ESTADUAL DE MARINGA; INDUSTRIAL A...,CLEAN WEINHEIM INTERNET
2,111305956227,4.0,,,,17,15.0,PADETEC PARQUE DE DESENVOLVIMENTO TECNOLOGICO ...,REVISTA DA ABPI; ASSOCIACAO ACADEMICA DE PROPR...
...,...,...,...,...,...,...,...,...,...
197,4515964877163,2.0,,,,49,35.0,UNIVERSIDADE FEDERAL RURAL DO SEMI ARIDO; UNIV...,UNIVERSIDADE FEDERAL RURAL DO SEMI ARIDO
198,4530949909455,4.0,,,,3,29.0,UNIVERSIDADE DE CAXIAS DO SUL; INSTITUTO DE CR...,UNIVERSIDADE FEDERAL DE SANTA MARIA
199,4597502232092,2.0,,,,107,27.0,UNIVERSIDADE ESTADUAL DE CAMPINAS; REVISTA DA ...,ORAL DISEASES 1354 523X


# IMPORT CSV _ IES

In [8]:
# LISTA DE IES NACIONAIS
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

df_uni = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_filtered = df_uni[['NOME_DA_IES', 'SIGLA']]
df_uni_filtered = df_uni_filtered.fillna('')
df_uni_filtered = df_uni_filtered.sort_values(['NOME_DA_IES'], ascending=False)
df_uni_filtered

,NOME_DA_IES,SIGLA
862,iCEV - Instituto de Ensino Superior,
2205,VOAS - INSTITUTO DE APRENDIZAGEM SUPERIOR,VOAS
2873,Universidade do Estado de Mato Grosso Carlos A...,UNEMAT
...,...,...
30,ACADEMIA DE POLÍCIA MILITAR DE MINAS GERAIS,APM
3933,ACADEMIA DA FORÇA AÉREA,AFA
2106,ABEU - CENTRO UNIVERSITÁRIO,UNIABEU


# PRE-PROCESSAMENTO

In [9]:
def clean_names(df, column_name):

    # Remover '&'
    df[column_name] = df[column_name].replace('&AMP;', ' ', regex=True)

    # Remover todos os caracteres nao alfa-numericos, exceto whitespace
    df[column_name] = df[column_name].str.replace('[^\w\s]', ' ', regex=True)

    # Normalizar a string
    df[column_name] = df[column_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Remover stopwords
    stop = ['DO', 'DA', 'DE', 'E']
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Strip whitespace
    df[column_name] = df[column_name].apply(lambda x: x.strip())

    return df

In [13]:
df2 = df2.apply(lambda x:x.astype(str).str.upper())
df2 = clean_names(df2, 'ListaFiliacoes')
df2 = clean_names(df2, 'UltimaFiliacao')

df_uni_filtered = df_uni_filtered.apply(lambda x:x.astype(str).str.upper())
df_uni_filtered = clean_names(df_uni_filtered, 'NOME_DA_IES')

In [14]:
# Split lista de IES por inicio do nome
df_uni_filtered.sort_values(by=["NOME_DA_IES"], ascending=True)

df_uni_outros = df_uni_filtered.copy(deep=True)

df_uni_instituto = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')].index, inplace=True)

df_uni_faculdades = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')].index, inplace=True)

df_uni_escola = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')].index, inplace=True)

df_uni_universidade = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')].index, inplace=True)

df_uni_instituicao = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')].index, inplace=True)

# List
# df_uni_outros, df_uni_instituto, df_uni_faculdades, df_uni_escola, df_uni_universidade, df_uni_instituicao

# PLN

In [19]:
import re

def get_uni_list_df(nome_instituicao):
    if 'INSTITUTO' in nome_instituicao:
        return df_uni_instituto, tfidf_matrix_instituto
    elif 'FACULDADE' in nome_instituicao:
        return df_uni_faculdades, tfidf_matrix_faculdades
    elif 'ESCOLA' in nome_instituicao:
        return df_uni_escola, tfidf_matrix_escola
    elif 'UNIVERSIDADE' in nome_instituicao:
        return df_uni_universidade, tfidf_matrix_universidade
    elif 'INSTITUICAO' in nome_instituicao:
        return df_uni_instituicao, tfidf_matrix_instituicao

    return df_uni_outros, tfidf_matrix_outros


def iterate_sigla(nome_instituicao, df_for_search):

    matched_rows = []

    for index, row in df_for_search.iterrows():

        if row['SIGLA']:

            pattern = r'\b' + row['SIGLA'] + r'\b'
            if re.search(pattern, nome_instituicao):
                matched_rows.append(row['SIGLA'])
                print("Matched SIGLA: ", row['SIGLA'], " - ", row['NOME_DA_IES'])

    return ",".join(str(element) for element in matched_rows)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vetorização TF-IDF
vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_DA_IES']) # geral
tfidf_matrix_instituto = vectorizer.transform(df_uni_instituto['NOME_DA_IES'])
tfidf_matrix_faculdades = vectorizer.transform(df_uni_faculdades['NOME_DA_IES'])
tfidf_matrix_escola = vectorizer.transform(df_uni_escola['NOME_DA_IES'])
tfidf_matrix_universidade = vectorizer.transform(df_uni_universidade['NOME_DA_IES'])
tfidf_matrix_instituicao = vectorizer.transform(df_uni_instituicao['NOME_DA_IES'])
tfidf_matrix_outros = vectorizer.transform(df_uni_outros['NOME_DA_IES'])

# Nova Coluna com Resultado
df2['nome_corrigido'] = ''
df2['sigla_detectado'] = ''

In [20]:
# Ultima Filiacao

threshold = 0.8

for index, row in df2.iterrows():

    nome_instituicao = row['UltimaFiliacao']

    if nome_instituicao:

        print(nome_instituicao)

        # Ver se contém a sigla
        sigla_detectado = iterate_sigla(nome_instituicao, df_uni_filtered)
        if sigla_detectado:
            df2.at[index, 'sigla_detectado'] = sigla_detectado

        # Se nao contem a sigla, procurar por nome completo
        else:
            vector_comparacao = vectorizer.transform([nome_instituicao])

            uni_list_df, tfidf_matrix_df = get_uni_list_df(nome_instituicao)

            similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

            max_value = np.max(similaridade)
            max_index = np.argmax(similaridade)

            nome_corrigido = uni_list_df.iloc[max_index]['NOME_DA_IES']

            print("nome corrigido : ", nome_corrigido, max_value)

            if (max_value >= threshold):
                df2.at[index, 'nome_corrigido'] = nome_corrigido
            else:
                df2.at[index, 'nome_corrigido'] = ''

        print("==================")

INSTITUTO ASTRONOMIA GEOFISICA CIENCIAS ATMOSFERICAS UNIVERSIDADE
nome corrigido :  INSTITUTO SUPERIOR CIENCIAS SAUDE 0.4855559789414471
CLEAN WEINHEIM INTERNET
nome corrigido :  UGV CENTRO UNIVERSITARIO 0.0
REVISTA ABPI ASSOCIACAO ACADEMICA PROPRIEDADE INTELECTUAL CONSELHO NACIONAL DESENVOLVIMENTO CIENTIFICO TECNOLOGICO LIVROS PROFNIT
nome corrigido :  CENTRO TECNOLOGICO POSITIVO 0.3168343449231602
CONSELHO NACIONAL DESENVOLVIMENTO CIENTIFICO TECNOLOGICO CHEMICAL ENGINEERING RESEARCH AMP DESIGN
nome corrigido :  CENTRO TECNOLOGICO POSITIVO 0.31980342655620914
INDIAN JOURNAL OF PHARMACEUTICAL SCIENCES
nome corrigido :  LINK SCHOOL OF BUSINESS 0.53726589073669
UNIVERSIDADE CANDIDO MENDES
nome corrigido :  UNIVERSIDADE CANDIDO MENDES 1.0000000000000002
REVISTA BRASILEIRA ENFERMAGEM
nome corrigido :  UGV CENTRO UNIVERSITARIO 0.0
PETROLEO BRASILEIRO SA INSTITUTO DOM BOSCO
Matched SIGLA:  DOM BOSCO  -  FACULDADE DOM BOSCO MARINGA
Matched SIGLA:  DOM BOSCO  -  FACULDADE DOM BOSCO
UNIVERSIDAD